# On-policy Prediction with Approximation

Approximate value function $v_\pi$ with $\hat{v}(s, \mathbf{w}) \approx v_\pi(s)$, where $\mathbf{w} \in \mathbb{R}^d$, and $d \ll |\mathscr{S}|$. Because we cannot memorize a value for every state, the approximate value function must learn to _generalize_ and learn the value of previously unseen states.

Learning a value function makes RL techniques also apply to partially observable problems (POMDPs), since the features available to the value function may only consist of the observable features.

However, a value function approximator does not guarantee that the history of the state is accounted for.

# 9.1 Value-function Approximation

Individual update $s \mapsto u$. Use supervised learning using update targets $\{ s_i \mapsto u_i \}$ as a dataset.

Must be able to learn incrementally, and handle a moving target since $\pi$ changes with $q_\pi$

# 9.2 The Prediction Objective ($\mathrm{\overline{VE}}$)

Value function updates affect the prediction for multiple states.

It may not be possible to exactly predict the value of every state.

Assuming undercapacity to learn exact value function.

Specify a distribution over states $\mu(s)$ $\sum_s{\mu(s)} = 1$


_Mean squared value error_ : $\mathrm{\overline{VE}}$

$$
\begin{align}
\mathrm{\overline{VE}}(\mathbf{w})& \; \dot{=} \; \mathbb{E}_{S\sim\mu}\left[\left(v_\pi(S) - \hat{v}(S, \mathbf{w})\right)^2\right]\\
&= \sum_{s\in\mathscr{S}}{\mu(s)\left(v_\pi(s) - \hat{v}(s, \mathbf{w})^2\right)}
\end{align}
$$

$\sqrt{\mathrm{\overline{VE}}}$ is a measure of the typical deviation between the true value and the approximated value. $\mu(s)$ is typically chosen to be or to approximate the fraction of time the agent reaches $s$, and is called the _on-policy distribution_.

**The on-policy distribution in episodic tasks**

Let the initial state distribution be $S_0 \sim h$ and $\eta(s)$ be the average number of time steps spent in state $s$ in a single episode.

$$
\eta(s) = h(s) + \sum_{s'\in\mathscr{S}, a\in\mathscr{A}(s')}\eta(s')p(s\mid a, s') \pi(a\mid s')
$$

This can be thought of as a dynamic programming equation for the number of visits to a state $s$. It can be solved and normalized to give $\mu$

$$
\mu(s) = \frac{\eta(s)}{\sum_{s'}{\eta(s')}}
$$

Discounting can be added by multiplying the second term of the recurrence equation for $\eta(s)$ by $\gamma$.

In continuing tasks, the on-policy distribution is the stationary distribution of states under policy $\pi$.

$$
\mu(s) = \sum_{s'\in\mathscr{S},a\in\mathscr{A}(s')}{\mu(s')\pi(a\mid s')p(s \mid s', a)}
$$

Ideally, we would want to find a global optimum for $\mathrm{\overline{VE}}$, that is, a weight vector $\mathbf{w}^*$ such that:

$$
\mathrm{\overline{VE}}(\mathbf{w}^*) \le \mathrm{\overline{VE}}(\mathbf{w}) \quad \mathrm{for\,all} \; \mathbf{w} \in \mathbb{R}^d
$$

This is not always possible for complex hypothesis classes. We can however find _local_ optima for these cases. No guarantee for convergence to a global optimum, or even within a bound of the optimum. Can even diverge.

Not all function approximation schemes can be considered here, so we limit ourselves to gradient based methods, particularly linear ones.

## Stochastic-gradient and Semi-gradient Methods

Consider a fixed size weight vector $\mathbf{w} = \begin{pmatrix} w_1 & w_2 & w_3 & \dots & w_d\end{pmatrix}^T$. Suppose further that $v(s, \mathbf{w})$ is a differentiable function of $\mathbf{w}$ at all states $s$. We will be incrementally updating an estimate of $\mathbf{w}$, starting at some initial estimate $\mathbf{w}_0$, in order to form the sequence $\mathbf{w}_t$ for $t = 0, 1, 2, \dots$. We observe a new sample state $S_t$ on each time step, and it's true value under the current policy. To select $\mathbf{w}$, we perform stochastic gradient descent as follows:

$$
\begin{align}
\mathbf{w}_{t+1} & \; \dot{=} \; \mathbf{w}_t - \frac{1}{2}\alpha\nabla_\mathbf{w}\left[v_\pi(S_t) - \hat{v}(S_t, \mathbf{w_t})\right]^2\\
&= \mathbf{w}_t + \alpha\left(v_\pi(S_t) - \hat{v}(S_t, \mathbf{w}_t)\right)\nabla_{\mathbf{w}}\hat{v}(S_t, \mathbf{w}_t)
\end{align}
$$

where $\nabla_\mathbf{w} f(\mathbf{w}) = \begin{pmatrix} \frac{\partial f}{\partial w_1} & \frac{\partial f}{\partial w_2} & \dots & \frac{\partial f}{\partial w_d} \end{pmatrix}\$

We do not attempt zero out the error on the example since we need to balance other states. Additionally, there is no guarantee that the error on this particular state will be made zero by moving in the gradient direction.

Stochastic descent only converges when the step-size decreases over time such that:

$$
\sum_t{\alpha_t} = \infty \quad \sum_t{\alpha_t^2} = 0
$$

Now consider the case where $v_\pi(s)$ is unavailable, and instead only one of the bootstrap targets, or an otherwise corrupted form of $v_\pi(s)$, is available. We call this value $U_t$ as in the earlier section. By substituting $v_\pi(s)$ with $U_t$, we get the update step for gradient based value estimation:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t + \left(U_t - \hat{v}(S_t, \mathbf{w}_t)\right)\nabla_{\mathbf{w}}{\hat{v}(S_t, \mathbf{w})}
$$

If we select $U_t = G_t$, we get _Gradient Monte-Carlo Value Estimation_.

Note however that if we were to have used a bootstrapped estimate such as:

$$
U_t = R_t + \gamma \hat{v}(S_{t+1}, \mathbf{w})
$$

will depend on the current value of $\mathbf{w}$ and therefore be _biased estimators_ of $v_\pi$, and so convergence is not guaranteed. Therefore, gradient bootstrapping methods are _not_ technically gradient descent methods, as our estimator is not guaranteed to have the true value of the gradient as its expectation. They account only for the participation of $\mathbf{w}$ on the right hand side of the difference, but not for the update in the target. Therefore, they only include part of the gradient and are called _semi-gradient_ methods.

Despite this, semi-gradient methods still converge robustly, can be used on continuing problems, and converge more quickly than Monte-Carlo methods for the reasons discussed in Chapter 6.

### State aggregation

State aggregation assigns one estimated value (and thus a component of $\mathbf{w}$ to an entire collection of states. So long as the partitioning of states is computable, state aggregation can be used with stochastic gradient descent.

**Example: 1000 State Random 

In [28]:
function state_aggregation_gradient_mc(;size=1000, N=1, d=10, α=2e-5)
    𝒮 = 1:size
    bucket_size = size ÷ d
    V = zeros(d)
    for i in 1:N
        S = Int[]
        s = size ÷ 2
        println("iter $(i)")
        while s in 𝒮
            push!(S, s)
            lr = rand() < 0.5 ? -1 : 1
            sp = s + lr * rand(1:100)
            if sp > 1000
                println("End right")
                sp = 1001
                r = 1
            elseif sp < 1
                println("End left")
                sp = 0
                r = -1
            end
            s = sp
        end
            
        for s in reverse(S)
            V[s ÷ bucket_size] += α*(r - V[s ÷ bucket_size])
        end
    end
    
    return V
end

state_aggregation_gradient_mc (generic function with 1 method)

In [29]:
state_aggregation_gradient_mc()

iter 1
End left


LoadError: BoundsError: attempt to access 10-element Vector{Float64} at index [0]

## Linear Methods

We consider the case where $\hat{v}(s, \mathbf{w})$ is linear in $\mathbf{w}$. We assume that for each state there is a feature vector $\mathbf{x}(s) \in \mathbb{R}^d$. We can then define:

$$
\hat{v}(s, \mathbf{w})) = \mathbf{w}^\top\mathbf{x}(s)
$$

In this case the gradient expression is incredibly simple:

$$
\nabla_{\mathbf{w}}v(s, w) = \mathbf{x}(s)
$$

And so we have as our update:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t + \alpha\left(U_t - \mathbf{w}^\top\mathbf{x}(s)\right)\mathbf{x}(s)
$$

Guaranteed to converge to a _global_ optimum

> _Exercise 9.1_ Show that tabular methods such as presented in Part I of this book are a special case of linear function approximation. What would the feature vectors be?

It is a special case of linear function approximation where, for every state $s_i$ 

$$
x_i(s) = \begin{cases} 1 & s = s_i \\ 0 & \mathrm{otherwise} \end{cases}
$$

This works for cases where $|\mathscr{S}| < \infty$.

# 9.5 Feature Construction for Linear Methods

> _Exercise 9.2_ Why does (9.17) define $(n+1)^k$ distinct features for dimension $k$?

It defines $(n+1)^k$ features because a feature will be defined by a selection of exponents on each of the $k$ state variables. Since there are $n + 1$ available exponents (including zero), the total number of features is $(n + 1)^k$

> _Exercise 9.3_ What $n$ and $c_{i,j}$ produce the feature vectors $\mathbf{x}(s) = \begin{pmatrix} 1 & s_1 & s_2 & s_1 s_2 & s_1^2 & s_2^2 & s_1 s_2^2 & s_1^2 s_2 & s_1^2 s_2^2 \end{pmatrix}$

$n = 2$ and

$$
c = \begin{pmatrix} 0 & 0 \\ 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 2 & 0 \\ 0 & 2 \\ 2 & 2 \end{pmatrix}
$$

# 9.5.4 Tile Coding

> _Exercise 9.4_ Suppose we believe that one of two state dimensions is more likely to have an effect on the value function than is the other, that generalization should be primarily across this dimension than along it. What kind of tilings could be used to take advantage of this prior knowledge?

One could use tiles that are asymmetrically elongated along the dimension that we expect to have less of an effect. This increases the scale of generalization along that dimension and leads to the overall value of that feature being learned more quickly but with less precision.

# 9.5.5 Radial Basis Functions

Use as features $x_i(s) \;\dot{=}\; \exp\left(-\frac{\|s-c_i\|^2}{2\sigma_i}\right)$ for fixed centers $c_i$ and standard deviations $\sigma_i$.

# 9.6 Selecting Step-Size Parameters Manually

> _Exercise 9.6_ Suppose you are using tile coding to transform a seven-dimensional continuous state space into binary feature vectors to estimate a state value function $v(s,\mathbf{w}) \approx v_\pi(s)$. You believe that the dimensions do not interact strongly, so you decide to use eight tilings of each dimension separately (stripe tilings), for $7 \times 8 = 56$ tilings. In addition, in case there are some pairwise interactions between the dimensions, you also take all $\begin{pmatrix} 7 \\ 2 \end{pmatrix} = 21$ pairs of dimensions and tile each pair conjunctively with rectangular tiles. You make two tilings for each pair of dimensions, making a grand total of $21 \times 2 + 56 = 98$ tilings. Given these feature vectors, you suspect that you still have to average out some noise, so you decide that you want learning to be gradual, taking about 10 presentations with the same feature vector before learning nears its asymptote. What step-size parameter should you use? Why?

Since $\mathbf{x}(s)$ will have a component $x_j(s) = 1$ for each tiling, and there are 98 separate tilings, each vector will have $\|\mathbf{x}\|^2 = 98$. According to the rule (9.19), we select $\alpha$ as

$$
\begin{align}
\alpha &= \left(\tau\mathbb{E}[\mathbf{x}^\top\mathbf{x}]\right)^{-1}\\
&= \left(10 \cdot 98 \right)^{-1}\\
&= \frac{1}{980}
\end{align}
$$

> _Exercise 9.6_ If $\tau = 1$ and $\mathbf{x}(S_t)^\top\mathbf{x} = \mathbb{E}[\mathbf{x}^\top\mathbf{x}]$, prove that (9.19) together with (9.7) and linear function approximation results in the error being reduced to zero in one update.

Recall (9.7)

$$
\begin{align}
\mathbf{w}_{t+1} &= \mathbf{w}_t + \alpha\left[U_t - \hat{v}(S_t, \mathbf{w}_t)\right]\nabla\hat{v}(S_t, \mathbf{w}_t)\\\\
&= \mathbf{w}_t + \frac{1}{\tau\mathbf{x}^\top\mathbf{x}}\left[U_t - \mathbf{w}^\top\mathbf{x}\right]\mathbf{x}
\end{align}
$$

And compute:

$$
\begin{align}
\mathbf{w}_{t+1}^\top\mathbf{x} &= \mathbf{x}^\top\left(\mathbf{w}_t + \frac{1}{\tau\mathbf{x}^\top\mathbf{x}}\left[U_t - \mathbf{w}^\top\mathbf{x}\right]\mathbf{x}\right)\\\\
&= \mathbf{x}^\top\mathbf{w}_t + \frac{1}{\mathbf{x}^\top\mathbf{x}}\left[U_t - \mathbf{w}^\top\mathbf{x}\right]\mathbf{x}^\top\mathbf{x}\\\\
&= U_t
\end{align}
$$

In [1]:
X = randn(1000, 10)

1000×10 Matrix{Float64}:
  1.06247      1.56691    0.993008   …   0.282672    0.728993     1.03661
 -0.172078    -1.40331    0.0500425      1.7024     -0.518671    -1.56776
 -0.723685    -1.09834    2.5193        -0.757337   -1.50908     -0.251429
 -1.10623     -1.84087    1.77786        0.610207    1.54883     -2.35321
 -1.63846     -1.45454    0.795906      -0.221461   -0.205146    -0.496482
  0.173946     1.12732    1.21284    …  -0.344975    1.26145     -0.577963
 -0.0607236   -0.37007    0.654707       0.0395516  -0.842974    -0.339456
 -0.755839     0.459805  -0.500829      -0.157568   -0.154092    -0.702444
 -0.336127    -1.69283   -0.338961       1.81823     0.245318    -0.278454
  1.22818     -0.609106   0.25639        1.05115    -1.35119     -0.403135
 -0.537323     1.38656    0.856401   …  -1.53905    -1.25833     -0.297074
 -1.99471     -0.888106  -0.723359       0.755329   -0.882116    -1.645
 -1.17834     -1.45449    0.563329       0.095328    2.04182     -0.344908
  ⋮   

In [3]:
R = randn(1000)

1000-element Vector{Float64}:
 -0.7413358218242678
 -0.3203947098503924
 -0.48140262170152087
 -1.5267360760388147
 -1.7436763967986015
 -0.8605451423222329
  1.32637735764156
 -1.8047280121495013
  0.43683207355916137
  0.08778762585577046
 -1.2437287360314655
 -0.4942396453138762
 -0.3663208510228378
  ⋮
  0.5350302599163258
 -0.2502198399827746
 -1.3419662266422365
 -0.6153265711407515
 -0.8670341193371606
  0.9433348550481715
 -0.5672025680115018
 -1.1774948253913893
 -1.6475485890688273
  1.2028549061689007
  1.713733236479457
 -0.2570203866661604

In [12]:
Xp = vcat(X[2:1000,:], randn(10)')

1000×10 Matrix{Float64}:
 -0.172078    -1.40331    0.0500425  …   1.7024     -0.518671    -1.56776
 -0.723685    -1.09834    2.5193        -0.757337   -1.50908     -0.251429
 -1.10623     -1.84087    1.77786        0.610207    1.54883     -2.35321
 -1.63846     -1.45454    0.795906      -0.221461   -0.205146    -0.496482
  0.173946     1.12732    1.21284       -0.344975    1.26145     -0.577963
 -0.0607236   -0.37007    0.654707   …   0.0395516  -0.842974    -0.339456
 -0.755839     0.459805  -0.500829      -0.157568   -0.154092    -0.702444
 -0.336127    -1.69283   -0.338961       1.81823     0.245318    -0.278454
  1.22818     -0.609106   0.25639        1.05115    -1.35119     -0.403135
 -0.537323     1.38656    0.856401      -1.53905    -1.25833     -0.297074
 -1.99471     -0.888106  -0.723359   …   0.755329   -0.882116    -1.645
 -1.17834     -1.45449    0.563329       0.095328    2.04182     -0.344908
 -1.5602       1.35537    1.34052        1.15055    -0.91213     -0.710362
  ⋮  

In [ ]:
X * w ≈ R + γ*Xp*w

In [13]:
γ = 0.99

0.99

In [14]:
Xt = (X - γ*Xp) 

1000×10 Matrix{Float64}:
  1.23283     2.9562      0.943466   …  -1.4027      1.24248    2.5887
  0.54437    -0.315956   -2.44407        2.45216     0.97532   -1.31885
  0.371481    0.724124    0.759224      -1.36144    -3.04243    2.07825
  0.515843   -0.400876    0.989912       0.829453    1.75193   -1.86169
 -1.81066    -2.57059    -0.404806       0.120064   -1.45398    0.0757019
  0.234063    1.49369     0.564681   …  -0.384131    2.09599   -0.241902
  0.687557   -0.825277    1.15053        0.195544   -0.690422   0.355963
 -0.423073    2.1357     -0.165258      -1.95761    -0.396958  -0.426774
 -1.55202    -1.08981    -0.592786       0.777591    1.58299    0.12065
  1.76013    -1.9818     -0.591447       2.57481    -0.105444  -0.109031
  1.43744     2.26579     1.57253    …  -2.28682    -0.385031   1.33147
 -0.828159    0.55184    -1.28105        0.660955   -2.90352   -1.30354
  0.366266   -2.79631    -0.76379       -1.04371     2.94483    0.35835
  ⋮                               

In [22]:
Xt' * Xt \ (Xt' * R)

10-element Vector{Float64}:
 -0.013128026267483738
 -0.0016226516046837886
  0.001745169297961782
 -0.027207828847988714
  0.009408678268168017
  0.001101334936732768
 -0.04157798160540794
 -0.03810622092191169
 -0.036677485542707354
  0.006999798673187679

# 9.7 Nonlinear Function Approximation: Artificial Neural Networks

# 9.8 Least-Squares TD

Recall 

# 9.12 Summary

> _Exercise 9.7_ One of the simplest artificial neural networks consists of a single semi-linear unit with a logistic nonlinearity. The need to handle approximate value functions of this form is common in games that end with either a win or a loss, in which case the value of a state can be interpreted as the probability of winning. Derive the learning algorithm for this case, from (9.7), such that no gradient notation appears.

A single linear function following by the logistic linearity will have gradient:

$$
\begin{align}
\nabla_\mathbf{w}\sigma(\mathbf{w}^\top\mathbf{x}) &= \sigma'(\mathbf{w}^\top\mathbf{x})\nabla_\mathbf{w}\mathbf{w}^\top\mathbf{x} = \sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}\\
\end{align}
$$

Thus substituting $\hat{v}(s, \mathbf{w}) = \sigma(\mathbf{w}^\top\mathbf{x}(s))$, we have for our learning rule:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t + \alpha\left[U_t - \sigma(\mathbf{w}^\top\mathbf{x})\right]\sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}
$$

> _Exercise 9.8_ Arguably, the squared error used to derive (9.7) is inappropriate for the case treated in the preceding exercise, and the right error measure is the cross-entropy loss (which you can find on Wikipedia). Repeat the derivation in Section 9.3, using the cross-entropy loss instead of the squared error in (9.4), all the way to an explicit form with no gradient or logarithm notation in it. Is your final form more complex, or simpler, than that you obtained in the preceding exercise?

$$
\begin{align}
\mathbf{w}_{t+1} &= \mathbf{w}_t - \alpha\nabla\left[-U_t\log{\sigma(\mathbf{w}^\top\mathbf{x})} - (1 - U_t)\log{\sigma(-\mathbf{w}^\top\mathbf{x})}\right]\\\\
&=\mathbf{w}_t - \alpha\left[-U_t\nabla\log{\sigma(\mathbf{w}^\top\mathbf{x})} - (1-U_t)\nabla\log{\sigma(-\mathbf{w}^\top\mathbf{x})}\right]\\\\
&=\mathbf{w}_t - \alpha\left[-U_t\frac{\sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}}{\sigma(\mathbf{w}^\top\mathbf{x})} + (1-U_t)\frac{\sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}}{\sigma(-\mathbf{w}^\top\mathbf{x})}\right]\\\\
&=\mathbf{w}_t - \alpha\left[-U_t\frac{\sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}}{\sigma(\mathbf{w}^\top\mathbf{x})} + (1-U_t)\frac{\sigma(\mathbf{w}^\top\mathbf{x})(1 - \sigma(\mathbf{w}^\top\mathbf{x}))\mathbf{x}}{\sigma(-\mathbf{w}^\top\mathbf{x})}\right]
\end{align}
$$